# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [6]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [9]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
    df_da = -3*a**2 - 0.5*a**-0.5
    df_db = 3*cos(3*b) + 2.5*b**1.5
    df_dc = c**-2
    return [df_da, df_db, df_dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [32]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# numerical derivative follows definition of derivative:
# [f(x+h) - f(x)] / h
# For multivariate, I can hold the other two dims constant while adding to the third

def calc_grad(fxn, dim):
    x1 = [2, 3, 4]
    y1 = fxn(*x1)
    h = 0.000001
    x1[dim] += h
    y2 = fxn(*x1)
    return (y2 - y1) / h

# -----------
numerical_grad = [calc_grad(f, 0), calc_grad(f, 1), calc_grad(f, 2)]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [29]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

def calc_grad_2h(fxn, dim):
    h = 0.001
    x1 = [2, 3, 4]
    x1[dim] -= h
    y1 = fxn(*x1)
    x1[dim] += 2*h
    y2 = fxn(*x1)
    return (y2 - y1) / (2 * h)

# this formula is indeed much better. h can be 1000 times higher while still passing the test

# -----------
numerical_grad2 = [calc_grad_2h(f, 0), calc_grad(f, 1), calc_grad(f, 2)]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766
OK for dim 1: expected 10.25699027111255, yours returns 10.256990385926201
OK for dim 2: expected 0.0625, yours returns 0.06249999628948899


## section 2: support for softmax

In [69]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
      
    return out

    
  def __mul__(self, other):
      other = other if isinstance(other, Value) else Value(other)
      new_val = Value(self.data * other.data, (self, other), '*')
      def _backward():
          self.grad += other.data * new_val.grad
          other.grad += self.data * new_val.grad
      new_val._backward = _backward
      return new_val

    # d/dx of e^x = e^x. So in backprop, scale gradient by e^x as it passes through
  def exp(self):
      new_val = Value(exp(self.data), (self,), 'exp')
      def _backward():
          self.grad += new_val.data * new_val.grad
      new_val._backward = _backward
      return new_val

    # d/dx of log(x) = 1/x. So in backprop, divide gradient by x as it passes through
  def log(self):
      new_val = Value(log(self.data), (self,), 'log')
      def _backward():
          self.grad += (1/self.data) * new_val.grad
      new_val._backward = _backward
      return new_val
      
  def __truediv__(self, other):
      other = other if isinstance(other, Value) else Value(other)
      return self * other**-1

  def __pow__(self, other):
      other = other if isinstance(other, Value) else Value(other)
      new_value = Value(self.data**other.data, (self, other), f'**{other.data}')
      def _backward():
          self.grad += other.data * self.data**(other.data-1) * new_value.grad
      new_value._backward = _backward
      return new_value
  def __neg__(self):
      return self * -1

  def __sub__(self, other):
      return self + -other

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()
  def __radd__(self, other):
      return self + other
      
  def __rtruediv__(self, other):
      return other * self**-1
      
  def __rmul__(self, other):
      return self * other

In [70]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [73]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

logits = torch.tensor([0, 3, -2, 1], dtype=torch.float32, requires_grad=True)
probs = torch.nn.functional.softmax(logits,dim=0)
loss = -probs[3].log()
loss.backward()
assert torch.allclose(torch.tensor(ans), logits.grad)